# TuningProcessSoftmax(Theory)

## 1.Tuning process

我们已经知道了神经网络有很多的超级参数:

- 学习率$\alpha$

- Momentum,$\beta_1$,$\beta_2$等

- #layers

- #hidden units

- 学习率衰减

- mini-batch size

- ...

其中相比之下比较重要的是$\alpha$,其次是$\beta$,#hidden layers, minibatch size等,那么我们现在就来看看如何系统组织的来调下参数,有时候一个神经网络会非常的大,超级参数也非常多,我们可能光是搜索超级参数就要搜索很多天/周.所以我们一般会启用多进程的方式(或者多线程,集群)的方式来快速搜索有效的超级参数.采用一种名为"Master-Works"的模式,works用于随机采取不同的参数值,比如进行loss的绘制和各种的信息统计,Master用于启动或者杀死works或者绘制works产生的数据.这样的话,多个works同时工作就能大大减少搜索参数的时间.

其实选择参数也有一些方式,下面讲两种方式:

经典搜索方式和Bergstra和Bengio在[Random Search for Hyper-Parameter Optimization](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf)论述的方式:

![](../../../picture/117.png)

**左图**就是经典的一种方式,在早一代的机器学习算法且数据量比较小的时候较流行,这个方式也很实用.但是在现代深度学习领域,我们更加倾向**右图**的方法,也就是随机取点,因为在深度学习中,随着网络的大小变化,超级参数也会越来越多,这样我们就很难知道哪一个参数是比较重要的.

举一个简单的例子,我们将图中的左侧看做是**parameter 1**,顶部看做是**parameters 2**.

那么在总量为9的情况下,左图的**parameters 1**实际上只有3中选择,而在右图随机选择下,我们会有9中选择.当然随着维数的增加,比如在三维的情况下就是在立方体中选择,右图可选择的情况就会远大于左图.

另外还有一种方式就是采取从粗糙到精细的策越,比如在二维情况下,当你发现某两个点的效果比较好,那么可以以这两个点适当增加范围来创建一个矩形,然后在这个矩形中重新随机选取点.

另外在[cs231n](http://cs231n.github.io/neural-networks-3/)中提到还有一种方式是贝叶斯超级参数优化,核心思想是在查询不同超级参数的性能时适当平衡探索-开发权衡.也有一些比较成熟的库[Spearmint](https://github.com/JasperSnoek/spearmint),[SMAC](http://www.cs.ubc.ca/labs/beta/Projects/SMAC/),[Hyperopt](http://jaberg.github.io/hyperopt/),但是如果对象是CNN的话使用这种精心选择的方式还是很难打败随机搜索的方式.

Ng给出的建议是多看看别人的网络架构的超级参数,因为那些是别人经过多次调解后比较优秀的参数.

另外在这里附上[ROC与AUC](https://www.jianshu.com/p/c61ae11cc5f6)

[PDF](../../../PDFS/C2W3L01notes.pdf)

### 2. Using an appropriate scale to pick hyperparameters

上面我们已经知道随机取值的有效性,但是我们并不是在有效范围内线性均匀取值,而是选择适当的尺标.

我们来看看一个关于学习率$\alpha$的例子:

假设你怀疑学习率$\alpha=0.0001-1$之间,由于我们电脑生成的是**伪随机数**,如果采用均匀取值,那么大的区间大约有90%的概率会落在0.1-1之间,也就是说0.1-1之间的数所占资源为90%,而0.0001-1之间所占用的资源只有10%左右.当然这是很不合理的,我们来看看下面这段代码:

In [1]:
import numpy as np
import random
count = 0
for i in range(1000):
    a = np.random.randint(1,10000) /10000
    if 0.1<a<1:
        count += 1
print('per:'+str((count/1000)*100)+'%')

per:89.4%


这样也就检验了我们不能随机均匀取值.

那么我们应该使用$log_{10}$来进行标尺

比如一个简单的例子:

```r=np.random.uniform(-4,0)```----> $[-4,0]$,其中uniform表示从均匀分布中抽取样本.

```alpha=10**r```----> $[0.0001,1]$

更普遍的是:

假设在范围$10^a,...,10^b$

$a=log_{10}(0.0001)=-4$

$b = log_{10}(1)=0$

那么我们就要在区间$[a,b]$内均匀取值,也就是$r\in [a,b]$,$\alpha=10^{r}$

当然我们也可以使用以上方式用于Regularization.

也就是说在小范围内,电脑参数的伪随机数不会大概率的落在某一个固定区间.

实际上一些Optimizer的超级参数$\beta$也是同样的方式,

比如你怀疑$\beta=0.9-0.999$之间的某一个数,那么也可以使用$log_{10}$的方式:

我们可以使用$1-\beta$来探究(主要计算方便)

$1-\beta=0.1-0.001$

那么$r\in[-3,-1]$,取完随机数```r=np.random.uniform(-3,-1)```--->$[-3,-1]$后

$1-\beta=10^r$

这里需要注意的是$\beta$一定不能使用线性均匀分布取值,**因为$\beta$接近1的时候会十分敏感:**

$β: 0.9000 ——> 1/(1-β)  \cong  10,\beta: 0.9005  ——> 1/(1-β)  \cong  10$

$β: 0.999 ——> 1/(1-β) \cong  1000, \beta:0.9995  ——> \cong  2000$

可以看出当β接近于1的时候即使很小的变化也会带来翻几倍的平均样本数量,所以像这种极度接近1的超级,我们一定要使用尺标的形式.

最后,如果数据量较大的话,那么使用均匀随机取值实际上得到的效果也是不错的.而且也是可以使用[交叉验证的方式](https://baike.baidu.com/item/%E4%BA%A4%E5%8F%89%E9%AA%8C%E8%AF%81)来检验参数的可行性.

[PDF](../../../PDFS/C2W3L02notes.pdf)

### Softmax

到目前为止我们所学的的都是一些二分类神经网络,但是如果有更多分类情况该怎么办呢?

有一种Logistics Regression的一般形式叫softmax Regression,可以作为一种多分类,关于更多softmax推导请看[softmax(Theory)](https://github.com/woaij100/Classic_machine_learning/blob/master/4-4%20Softmax(Theory).ipynb).这里就不在多赘述了.在这里我们主要看看softmax如何使用

假设你不单单只是要识别猫,你还要识别狗,识别小鸡,那么我们可以把猫设为类1,狗设为类2,小鸡设为类3,其他的设为0(在这里是考拉).

![](../../../picture/118.png)

我们会使用C来表示你输入将要分类的总个数$C= \#classes = 4  (0,...,3) = (0,...C-1)$

那么在下面这个神经网络,我们将会输出4中类别的预测值概率:

![](../../../picture/119.png)

因为实际上我们的$y$标签是一个$[1,4]$的形式,其中这里的4是类别数C.并且这4个概率和即:$P(other|x)+P(cat|x)+P(dog|x)+P(baby chicken|x)=1$

所以我们给出如下softmax layer:

![](../../../picture/120.png)

像往常一样,你会计算各层的线性部分,在Layer L,我们使用(features,n_sample)的形式:

(1) $Z^{[L]} = W^{[L]}a^{[L-1]} + b^{[L]}$,shape==(4,1),m组样本即使(4,m) 

(2) Activation Function(Softmax)

(3) $t = e^{Z^{[L]}}$零时变量shape == (4,1),m个样本即使(4,m) 

(4) $a^{[L]} = \frac{e^{Z^{[L]}}}{\sum_{j = 1}^{t}t_i}$t的标准化 shape == (4,1),m个样本即使(4,m),也就是为了其概率为1.

例子:

$Z^{[L]} = \begin{bmatrix}
5\\ 
2\\ 
-1\\
3\\ \end{bmatrix}$

$t = \begin{bmatrix}
e^{5}\\ 
e^{2}\\ 
e^{-1}\\
e^{3}\\ \end{bmatrix} = \begin{bmatrix}
148.4\\ 
7.4\\ 
0.4\\
20.1\\ \end{bmatrix}$

$a^{[L]} = \frac{t}{176.3} = \begin{bmatrix}
0.842\\ 
0.042\\ 
0.002\\
0.114\\ \end{bmatrix}$

所以这个例子中它属于其他类(classes = 0)的可能性最高

这里激活函数与之前学过的唯一不同的地方在于这里激活函数输入的是(C,1)输出的也是(C,1)维的向量(因为我们是做二分类(C=1,1),每次一个预测样本输出只能是1个)另外,m个样本即使(C,m).

**Softmax也是线性决策边界**

假设我们将样本特征$x_1,x_2$不经过隐藏层直接作用softmax,那么$z^{[1]} = w^{[1]} + b^{[1]},C=3$

由下面的图可以看出softmax是线性决策边界

![](../../../picture/121.png)

[PDF](../../../PDFS/C2W3L08notes.pdf)